# Tutorial 7 : Neural Process Graphs

Last Update : 28 July 2019

**Aim**: 


In [1]:
N_THREADS = 12
# Nota Bene : notebooks don't deallocate GPU memory
IS_FORCE_CPU = True # can also be set in the trainer

## Environment

In [2]:
cd ..

/conv


In [3]:
%autosave 600
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# CENTER PLOTS
from IPython.core.display import HTML
display(HTML(""" <style> .output_png {display: table-cell; text-align: center; margin:auto; }
.prompt display:none;}  </style>"""))

import os
if IS_FORCE_CPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = ""
    
import sys
sys.path.append("notebooks")

import numpy as np
import matplotlib.pyplot as plt
import torch
torch.set_num_threads(N_THREADS)

Autosaving every 600 seconds


# Dataset 

In [10]:
from torch_geometric.datasets import TUDataset
from sklearn.preprocessing import StandardScaler

In [11]:
def print_dataset(dataset):
    num_nodes = num_edges = 0
    for data in dataset:
        num_nodes += data.num_nodes
        num_edges += data.num_edges

    print('Name', dataset)
    print('Graphs', len(dataset))
    print('Nodes', num_nodes / len(dataset))
    print('Edges', (num_edges // 2) / len(dataset))
    print('Features', dataset.num_features)
    print('Classes', dataset.num_classes)
    print(dataset[0])
    print()

In [12]:
datasets=dict(enzymes = TUDataset(root='data/ENZYMES', name='ENZYMES', use_node_attr=True, ),
             proteins = TUDataset(root='data/PROTEINS_full', name='PROTEINS_full', use_node_attr=True),
             synthie = TUDataset(root='data/Synthie', name='Synthie', use_node_attr=True))

In [13]:
for d in datasets.values():
    print_dataset(d)
    print()

Name ENZYMES(600)
Graphs 600
Nodes 32.63333333333333
Edges 62.13666666666666
Features 21
Classes 6
Data(edge_index=[2, 168], x=[37, 21], y=[1])


Name PROTEINS_full(1113)
Graphs 1113
Nodes 39.05750224618149
Edges 72.8158131176999
Features 32
Classes 2
Data(edge_index=[2, 162], x=[42, 32], y=[1])


Name Synthie(400)
Graphs 400
Nodes 95.0
Edges 172.9275
Features 15
Classes 4
Data(edge_index=[2, 294], x=[90, 15], y=[1])




In [14]:
datasets=dict(enzymes = TUDataset(root='data/ENZYMES', name='ENZYMES', use_node_attr=True, ),
             proteins = TUDataset(root='data/PROTEINS_full', name='PROTEINS_full', use_node_attr=True),
             synthie = TUDataset(root='data/Synthie', name='Synthie', use_node_attr=True))

data_specific_kwargs = {k:dict(y_dim=d.num_node_features) for k,d in datasets.items()}

def train_test_split(d, transform=lambda x : StandardScaler().fit_transform(x)):
    if transform is not None:
        d.data.x = torch.from_numpy(transform(d.data.x.numpy()))
    d.shuffle()
    return list(d[:-len(d) // 10]), list(d[-len(d) // 10:])

# should store the split
datasets = {k:train_test_split(d) for k,d in datasets.items()}

In [15]:
X_DIM = 2  # 2D spatial input 
#Y_DIM = data.shape[0]
N_TARGETS = None#data.n_classes

#label_percentages = [N_TARGETS, N_TARGETS*2, 0.01, 0.05, 0.1, 0.3, 0.5, 1]

# Model

In [16]:
from skssl.transformers import GraphConvNeuralProcess, GraphNeuralProcessSSLLoss
from skssl.predefined import GCN, UnetGCN
from skssl.transformers.neuralproc.datasplit import precomputed_cntxt_trgt_split
from functools import partial
from torch_geometric.nn import GCNConv, GINConv
import torch.nn as nn

models = {}

r_dflt = 32
l_dflt = 5
s_dflt = True
train_eps_dflt = True
is_dc_dflt = False
btlnk_dflt = None
    
"""
for l in [5,7,9]:
    r = 32
    s = True
    train_eps = True
    is_dc = False
    btlnk = None
    
    # initialize one model for each dataset
    models["gcnp_gunet_l{}_r{}_s{}_traineps{}_dc{}_btlnk{}".format(l,r,s,train_eps, is_dc, btlnk)
          ] = partial(GraphConvNeuralProcess, 
                             r_dim=r,
                                Conv=lambda y_dim: GINConv(nn.Identity(), eps=1., train_eps=train_eps),
                              TmpSelfAttn=partial(UnetGCN, 
                                                  is_sum_res=s,
                                                  bottleneck=btlnk,
                                                  is_double_conv=is_dc,
                                                  Conv=partial(GCNConv, improved=True), 
                                                  max_nchannels=256, n_layers=l))

    
for r in [16,64]:
    l = l_dflt
    s = s_dflt
    train_eps = train_eps_dflt
    is_dc = is_dc_dflt
    btlnk = btlnk_dflt
    
    # initialize one model for each dataset
    models["gcnp_gunet_l{}_r{}_s{}_traineps{}_dc{}_btlnk{}".format(l,r,s,train_eps, is_dc, btlnk)
          ] = partial(GraphConvNeuralProcess, 
                             r_dim=r,
                                Conv=lambda y_dim: GINConv(nn.Identity(), eps=1., train_eps=train_eps),
                              TmpSelfAttn=partial(UnetGCN, 
                                                  is_sum_res=s,
                                                  bottleneck=btlnk,
                                                  is_double_conv=is_dc,
                                                  Conv=partial(GCNConv, improved=True), 
                                                  max_nchannels=256, n_layers=l))
    
for s in [False]:
    l = l_dflt
    train_eps = train_eps_dflt
    is_dc = is_dc_dflt
    btlnk = btlnk_dflt
    r = r_dflt
    
    # initialize one model for each dataset
    models["gcnp_gunet_l{}_r{}_s{}_traineps{}_dc{}_btlnk{}".format(l,r,s,train_eps, is_dc, btlnk)
          ] = partial(GraphConvNeuralProcess, 
                             r_dim=r,
                                Conv=lambda y_dim: GINConv(nn.Identity(), eps=1., train_eps=train_eps),
                              TmpSelfAttn=partial(UnetGCN, 
                                                  is_sum_res=s,
                                                  bottleneck=btlnk,
                                                  is_double_conv=is_dc,
                                                  Conv=partial(GCNConv, improved=True), 
                                                  max_nchannels=256, n_layers=l))
    
for train_eps in [False]:
    l = l_dflt
    s = s_dflt
    is_dc = is_dc_dflt
    btlnk = btlnk_dflt
    r = r_dflt
    
    # initialize one model for each dataset
    models["gcnp_gunet_l{}_r{}_s{}_traineps{}_dc{}_btlnk{}".format(l,r,s,train_eps, is_dc, btlnk)
          ] = partial(GraphConvNeuralProcess, 
                             r_dim=r,
                                Conv=lambda y_dim: GINConv(nn.Identity(), eps=1., train_eps=train_eps),
                              TmpSelfAttn=partial(UnetGCN, 
                                                  is_sum_res=s,
                                                  bottleneck=btlnk,
                                                  is_double_conv=is_dc,
                                                  Conv=partial(GCNConv, improved=True), 
                                                  max_nchannels=256, n_layers=l))
    
for is_dc in [True]:
    l = 10
    s = s_dflt
    train_eps = train_eps_dflt
    btlnk = btlnk_dflt
    r = r_dflt
    
    # initialize one model for each dataset
    models["gcnp_gunet_l{}_r{}_s{}_traineps{}_dc{}_btlnk{}".format(l,r,s,train_eps, is_dc, btlnk)
          ] = partial(GraphConvNeuralProcess, 
                             r_dim=r,
                                Conv=lambda y_dim: GINConv(nn.Identity(), eps=1., train_eps=train_eps),
                              TmpSelfAttn=partial(UnetGCN, 
                                                  is_sum_res=s,
                                                  bottleneck=btlnk,
                                                  is_double_conv=is_dc,
                                                  Conv=partial(GCNConv, improved=True), 
                                                  max_nchannels=256, n_layers=l))
    
for btlnk in ["channels"]:
    l = l_dflt
    s = s_dflt
    train_eps = train_eps_dflt
    is_dc = is_dc_dflt
    r = r_dflt
    
    # initialize one model for each dataset
    models["gcnp_gunet_l{}_r{}_s{}_traineps{}_dc{}_btlnk{}".format(l,r,s,train_eps, is_dc, btlnk)
          ] = partial(GraphConvNeuralProcess, 
                             r_dim=r,
                                Conv=lambda y_dim: GINConv(nn.Identity(), eps=1., train_eps=train_eps),
                              TmpSelfAttn=partial(UnetGCN, 
                                                  is_sum_res=s,
                                                  bottleneck=btlnk,
                                                  is_double_conv=is_dc,
                                                  Conv=partial(GCNConv, improved=True), 
                                                  max_nchannels=256, n_layers=l))
"""

# initialize one model for each dataset
models["transformer_gcnp_unetgcn"] = partial(GraphConvNeuralProcess, 
                         r_dim=r_dflt,
                            Conv=lambda y_dim: GINConv(nn.Identity(), eps=1., train_eps=train_eps_dflt),
                          TmpSelfAttn=partial(UnetGCN, 
                                              is_sum_res=s_dflt,
                                              bottleneck=btlnk_dflt,
                                              is_double_conv=is_dc_dflt,
                                              Conv=partial(GCNConv, improved=True), 
                                              max_nchannels=256, n_layers=l_dflt))

In [17]:
from utils.helpers import count_parameters
for k,v in models.items():
    print(k, "- N Param:", count_parameters(v(y_dim=3)))

transformer_gcnp_unetgcn - N Param: 42057


In [18]:
from skssl.transformers.neuralproc.datasplit import GridCntxtTrgtGetter, RandomMasker, no_masker, half_masker
from utils.data.tsdata import get_timeseries_dataset, SparseMultiTimeSeriesDataset

get_cntxt_trgt_test = GridCntxtTrgtGetter(context_masker=RandomMasker(min_nnz=0.01, max_nnz=0.50),
                                     target_masker=no_masker,
                                     is_add_cntxts_to_trgts=False)  # don't context points to tagrtes

get_cntxt_trgt_feat = GridCntxtTrgtGetter(context_masker=no_masker,
                                     target_masker=no_masker,
                                     is_add_cntxts_to_trgts=False)  # don't context points to tagrtes

get_cntxt_trgt = GridCntxtTrgtGetter(context_masker=RandomMasker(min_nnz=0.01, max_nnz=0.50),
                                 target_masker=RandomMasker(min_nnz=0.50, max_nnz=0.99),
                                 is_add_cntxts_to_trgts=False)  # don't context points to tagrtes

import torch
import skorch
from torch_geometric.data import Batch

def cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=False):
    def mycollate(data_list):
        
        if is_repeat_batch:
            data_list = data_list + data_list
            
        data = Batch.from_data_list(data_list, [])
        edge_attr = torch.ones_like(data.edge_index[0], dtype=torch.float) if data.edge_attr is None else data.edge_attr
        
        X, mask_context, mask_target = get_cntxt_trgt(data.x.t().unsqueeze(0), None, is_grided=True)
        data.x = X.squeeze(0).t()
            
        
        # Can't pass a Dataset directly, since it expects tensors. 
        # Use dict of tensors instead. Also, use torch.sparse for 
        # adjacency matrix to pass skorch's same-dimension check
        return {
            "X":{'x': data.x,
            'adj': torch.sparse.FloatTensor(data.edge_index, 
                                            edge_attr, 
                                            size=[data.num_nodes, data.num_nodes], 
                                            device=data.x.device),
            'batch': data.batch},
            'mask_context':mask_context.squeeze(0),
            'mask_target':mask_target.squeeze(),
            
        }, data.y
    
    return mycollate
        
class SkorchDataset(skorch.dataset.Dataset):
    def __init__(self, X, y):
        # We need to specify `length` to avoid checks
        super(SkorchDataset, self).__init__(X, y, length=len(X))
    
    def transform(self, X, y):
        return X   # Ignore y, since it is included in X
    

# Training

In [19]:
N_EPOCHS = 200 
BATCH_SIZE = 32
IS_RETRAIN = True # if false load precomputed
chckpnt_dirname="results/notebooks/neural_process_graph/"

from ntbks_helpers import train_models_

In [20]:
data_trainers = {}
data_trainers.update(train_models_({k:d for k,d in datasets.items() }, 
                       models,
                        partial(GraphNeuralProcessSSLLoss, is_clf=False),
                      data_specific_kwargs=data_specific_kwargs,
                     patience=15,
                     chckpnt_dirname=chckpnt_dirname,
                      max_epochs=N_EPOCHS,
                      batch_size=BATCH_SIZE,
                      is_retrain=IS_RETRAIN,
                      callbacks=[],
                      iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=True),  
                      iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_test),
                        dataset=SkorchDataset,
                      mode="transformer"))

"""
layers: 5 is best
enzymes/gcnp_gunet_l5_r32_sTrue_trainepsTrue_dcFalse_btlnkNone best epoch: 52 val_loss: 23.861642837524414
enzymes/gcnp_gunet_l7_r32_sTrue_trainepsTrue_dcFalse_btlnkNone : 25.95114517211914
enzymes/gcnp_gunet_l9_r32_sTrue_trainepsTrue_dcFalse_btlnkNone best epoch: 35 val_loss: 26.83774757385254
proteins/gcnp_gunet_l5_r32_sTrue_trainepsTrue_dcFalse_btlnkNone best epoch: 67 val_loss: 16.346792221069336
proteins/gcnp_gunet_l7_r32_sTrue_trainepsTrue_dcFalse_btlnkNone best epoch: 28 val_loss: 25.480146252348984
"""

"""
r dim: 32 is best
enzymes/gcnp_gunet_l5_r32_sTrue_trainepsTrue_dcFalse_btlnkNone best epoch: 52 val_loss: 23.861642837524414
proteins/gcnp_gunet_l5_r32_sTrue_trainepsTrue_dcFalse_btlnkNone best epoch: 67 val_loss: 16.346792221069336
enzymes/gcnp_gunet_l5_r16_sTrue_trainepsTrue_dcFalse_btlnkNone best epoch: 56 val_loss: 24.114784240722656
enzymes/gcnp_gunet_l5_r64_sTrue_trainepsTrue_dcFalse_btlnkNone best epoch: 48 val_loss: 26.669349670410156
proteins/gcnp_gunet_l5_r16_sTrue_trainepsTrue_dcFalse_btlnkNone best epoch: 71 val_loss: 17.402103424072266
proteins/gcnp_gunet_l5_r64_sTrue_trainepsTrue_dcFalse_btlnkNone best epoch: 55 val_loss: 18.61931386568722
"""

"""
sum : True is best
enzymes/gcnp_gunet_l5_r32_sFalse_trainepsTrue_dcFalse_btlnkNone best epoch: 21 val_loss: 27.104204177856445
proteins/gcnp_gunet_l5_r32_sFalse_trainepsTrue_dcFalse_btlnkNone best epoch: 70 val_loss: 19.60527303187457
"""

"""
train eps : True is best
enzymes/gcnp_gunet_l5_r32_sTrue_trainepsFalse_dcFalse_btlnkNone best epoch: 53 val_loss: 26.150896072387695
proteins/gcnp_gunet_l5_r32_sTrue_trainepsFalse_dcFalse_btlnkNone best epoch: 47 val_loss: 23.75955954962524
"""

"""
dc : False is best
enzymes/gcnp_gunet_l10_r32_sTrue_trainepsTrue_dcTrue_btlnkNone best epoch: 36 val_loss: 27.692407608032227
proteins/gcnp_gunet_l10_r32_sTrue_trainepsTrue_dcTrue_btlnkNone best epoch: 36 val_loss: 27.874570464365846
"""

"""
bottleneck : None is besst
enzymes/gcnp_gunet_l5_r32_sTrue_trainepsTrue_dcFalse_btlnkchannels best epoch: 46 val_loss: 26.540746688842773
proteins/gcnp_gunet_l5_r32_sTrue_trainepsTrue_dcFalse_btlnkchannels best epoch: 17 val_loss: 28.191771718871724
"""

"""
lr : best (5e-4 ?) 
5e-3 : enzymes/transformer_gcnp_unetgcn best epoch: 21 val_loss: 24.790180206298828
       proteins/transformer_gcnp_unetgcn epoch: 8 val_loss: 24.66300392150879
1e-3 : 25.6
    19.826078414916992
5e-4 : 25.889606475830078
    16.892475128173828
"""


--- Training enzymes/transformer_gcnp_unetgcn ---



  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        1.1331        1.0523     +  3.6402


      2        1.0841        1.0151     +  3.7022


      3        1.0464        0.9815     +  3.5548


      4        1.0115        0.9804     +  3.6696


...

    197        0.9688        1.0094        5.0135


    198        0.9738        0.9304     +  4.9004


    199        0.9891        0.9506        4.9785


    200        0.9884        1.0034        4.9458
Re-initializing module.
Re-initializing optimizer.
synthie/transformer_gcnp_unetgcn best epoch: 198 val_loss: 0.9303861260414124


'\nlr : best (5e-4 ?) \n5e-3 : enzymes/transformer_gcnp_unetgcn best epoch: 21 val_loss: 24.790180206298828\n       proteins/transformer_gcnp_unetgcn epoch: 8 val_loss: 24.66300392150879\n1e-3 : 25.6\n    19.826078414916992\n5e-4 : 25.889606475830078\n    16.892475128173828\n'

In [23]:
for k,t in data_trainers.items(): 
    for e, h in enumerate(t.history[::-1]):
        if h["valid_loss_best"]:
            print(k, "epoch:", len(t.history)-e, 
                  "val_loss:", h["valid_loss"])
            break

enzymes/transformer_gcnp_unetgcn epoch: 156 val_loss: 0.7818504571914673
proteins/transformer_gcnp_unetgcn epoch: 163 val_loss: 0.26717182993888855
synthie/transformer_gcnp_unetgcn epoch: 198 val_loss: 0.9303861260414124
